In [ ]:
# pip install scikit-surprise

import pandas as pd
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Load the MovieLens dataset
data = Dataset.load_builtin('ml-100k')

# Create a train-test split
trainset, testset = train_test_split(data, test_size=0.25)

# Create the SVD model
model = SVD()

# Train the model on the training set
model.fit(trainset)

# Make predictions on the test set
predictions = model.test(testset)

# Evaluate the model's performance
rmse = accuracy.rmse(predictions)
print("Root Mean Squared Error (RMSE):", rmse)

# Get recommendations for a specific user (user ID: 196)
user_id = str(196)
user_items = trainset.ur[trainset.to_inner_uid(user_id)]
items_already_rated = [trainset.to_raw_iid(item_id) for (item_id, _) in user_items]

# Get top 5 movie recommendations for the user
all_movie_ids = list(range(1, 1683))  # Assuming there are 1682 movies in the dataset
unrated_movie_ids = set(all_movie_ids) - set(items_already_rated)

# Predict ratings for unrated movies
unrated_movie_predictions = [(movie_id, model.predict(user_id, movie_id).est) for movie_id in unrated_movie_ids]

# Sort movie recommendations based on predicted ratings
sorted_movie_recommendations = sorted(unrated_movie_predictions, key=lambda x: x[1], reverse=True)
top_5_recommendations = sorted_movie_recommendations[:5]

# Display top 5 movie recommendations for the user
print(f"\nTop 5 movie recommendations for User {user_id}:")
for movie_id, predicted_rating in top_5_recommendations:
    movie_title = trainset.to_raw_iid(movie_id)
    print(f"Movie ID: {movie_id}, Movie Title: {movie_title}, Predicted Rating: {predicted_rating:.2f}")
